#### PROFILE THE DATASET FROM `GIZMOBOX_DEV.BRONZE.CUSTOMERS_VW`
- CHANGE DATATYPE FOR `CREATED_TIMESTAMP`
- CHANGE DATATYPE FOR `DATE_OF_BIRTH`
- CHANGE DATATYPE FOR `MEMBER_SINCE`

%md
#### TRANSFORM `CUSTOMERS` DATA
- 1. CONVERT `created_timestamp` TO TIMESTAMP DATATYPE

In [0]:
%sql
SELECT
CAST(created_timestamp AS TIMESTAMP) AS created_timestamp,
customer_id,
customer_name,
CAST(date_of_birth AS DATE) AS date_of_birth,
email,
CAST(member_since AS DATE) AS member_since,
telephone,
file_path,
file_name,
file_modification_time
FROM
(
    SELECT DISTINCT *
    ,RANK() OVER(PARTITION BY customer_id ORDER BY created_timestamp DESC) AS rnk
    FROM gizmobox_dev.bronze.customers_vw
                                    WHERE customer_id IS NOT NULL
                                    ORDER BY customer_id ASC) 
WHERE rnk = 1;

#### LOAD TRANSFORMED DATA INTO SILVER SCHEMA AS DELTA TABLE

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gizmobox_dev.silver.customers
AS
SELECT
CAST(created_timestamp AS TIMESTAMP) AS created_timestamp,
customer_id,
customer_name,
CAST(date_of_birth AS DATE) AS date_of_birth,
email,
CAST(member_since AS DATE) AS member_since,
telephone,
file_path,
file_name,
file_modification_time
FROM
(
    SELECT DISTINCT *
    ,RANK() OVER(PARTITION BY customer_id ORDER BY created_timestamp DESC) AS rnk
    FROM gizmobox_dev.bronze.customers_vw
                                    WHERE customer_id IS NOT NULL
                                    ORDER BY customer_id ASC) 
WHERE rnk = 1;

In [0]:
%sql
SELECT * FROM gizmobox_dev.silver.customers LIMIT 10

In [0]:
%sql
DESC EXTENDED gizmobox_dev.silver.customers

In [0]:
dbutils.notebook.exit('CUSTOMERS TABLE HAS BEEN LOADED IN SILVER SCHEMA....')